In [1]:
import requests
import polars as pl
import pandas as pd
from pathlib import Path
import geopandas as gpd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

from datetime import datetime

In [2]:
base = Path.home() / "Local_Workspace" / "Datasets" / "ARPA" / "TRENTINO" / "bolzano"

## API Bolzano
Stazioni recuperabili tramite API OpenData. Solo dal 2014.

In [3]:
sensors = pl.read_json(base / "api" / "sensors.json").with_columns(
    pl.col("SCODE").str.to_uppercase().str.strip_chars()
)
sensors.write_csv(base / "api" / "sensors.csv")
stations = gpd.read_file(base / "api" / "stations.geojson")
stations["SCODE"] = stations["SCODE"].str.upper().str.strip()

In [4]:
def query_payload(station_code, from_date: datetime, to_date: datetime):
    return {
        "station_code": station_code,
        "sensor_code": "LT",
        "date_from": from_date.strftime(r"%Y%m%d"),
        "date_to": to_date.strftime(r"%Y%m%d"),
        "output_format": "CSV",
    }

In [5]:
stat_ids = sensors.filter(pl.col("TYPE").eq("LT"))["SCODE"].to_list()

In [8]:
from time import sleep
from random import uniform


def file_path(station_id, year):
    path = base / "api" / "fragments" / f"{station_id}" / f"{year}.csv"
    if not path.parent.exists():
        path.parent.mkdir(parents=True)
    return path


def get_year_data(station_id, year):
    try:
        r = requests.get(
            "http://daten.buergernetz.bz.it/services/meteo/v1/timeseries",
            params=query_payload(
                station_id, datetime(year, 1, 1), datetime(year + 1, 1, 1)
            ),
        )
        if r.status_code != 200:
            sleep(uniform(0.2, 1))
            raise
        return r.text
    except:
        print(f"There was an error: {r.status_code}. Continuing...")
        return False


def get_station_data(station_id, bar: tqdm):
    for year in range(2010, 2025):
        path = file_path(station_id, year)
        if path.exists():
            bar.update()
            continue
        data = get_year_data(station_id, year)
        if not data:
            bar.update()
            continue
        if data.strip() == "":
            bar.update()
            continue
        with open(path, "wt") as file:
            file.write(data)
        bar.update()
        sleep(uniform(0.5, 1.5))
    bar.reset(total=15)

In [ ]:
for station_id in tqdm(stat_ids):
    with tqdm(total=15, leave=True) as bar:
        get_station_data(station_id, bar)

In [41]:
for file in list((base / "api" / "fragments").glob("**/*.csv"))[:1]:
    data = pl.read_csv(file, try_parse_dates=False).with_columns(
        pl.col("DATE")
        .str.replace("CET", "+0100")
        .str.replace("CEST", "+0200")
        .str.to_datetime(format=r"%Y-%m-%dT%H:%M:%S%z")
        .dt.convert_time_zone(time_zone="CET")
    )

In [10]:
for file in (base / "api" / "fragments").glob("**/*.csv"):
    station_id = file.parent.stem
    file_path = base / "api" / "dataset" / f"{station_id}" / f"{file.stem}.parquet"
    if not file_path.parent.exists():
        file_path.parent.mkdir(parents=True)
    pl.read_csv(file, try_parse_dates=False).with_columns(
        pl.col("DATE")
        .str.replace("CET", "+0100")
        .str.replace("CEST", "+0200")
        .str.to_datetime(format=r"%Y-%m-%dT%H:%M:%S%z")
        .dt.convert_time_zone(time_zone="CET"),
        pl.lit(station_id).str.to_uppercase().str.strip_chars().alias("original_id"),
    ).write_parquet(file_path)

## XLSX Data
Sempre da OpenData ma in formato tabelle excel

In [11]:
page = (base / "xlsx" / "xlsx_datastore.html").read_text()
landing_page = BeautifulSoup(page, "html.parser")

In [12]:
def li_class_resource_item(li):
    return li.has_attr("class") and "resource-item" in li["class"]


ids = [tag["data-id"] for tag in landing_page.find_all(li_class_resource_item)]

In [13]:
import re

files = list(map(lambda m: m.group(0), re.finditer(r"http://[^\"]+\.xlsx", page)))

In [ ]:
import urllib

with requests.Session() as session:
    for url in tqdm(files):
        r = session.get(url)
        with open(
            base / "xlsx" / urllib.parse.unquote(url.split("/")[-1]), "wb"
        ) as file:
            file.write(r.content)

In [16]:
def payload(data_id, n, offset):
    return {
        "resource_id": data_id,
        "limit": n,
        "offset": offset,
    }


def get_data(data_id, n, offset):
    return requests.get(
        "https://data.civis.bz.it/api/action/datastore_search",
        params=payload(data_id, n, offset),
    ).json()

In [11]:
excels_path = base / "xlsx"
xlsx_files = list(excels_path.glob("*.xlsx"))

In [12]:
def get_meta(path):
    # ETRS89/UTM32N
    coords = pd.read_excel(path, usecols=[7], skiprows=7, nrows=3, header=None)
    meta = coords.iloc[:, 0].str.split(" ").explode().iloc[[1, 3, 4]]
    meta.index = ["x", "y", "elevation"]
    return meta

# def alt_get_meta(path):
#     import openpyxl
#     ws = openpyxl.load_workbook(path, read_only=True)["Min-Max Extremwert-estremo"]
#     x = ws["J10"].value
#     y = ws["P10"].value
#     elevation = ws["BA10"].value
#     meta = pd.Series({"x": x, "y": y, "elevation": elevation}).str.split(" ").explode().iloc[[0, 2, 4]]
#     return meta

In [13]:
import re
reg = re.compile(r"-+")
meta_pattern = re.compile(r"^(?P<original_id>[A-Z0-9]+)-+(?P<station_name_de>[^-]+)-+(?P<station_name_it>.+)-multiannual-LT-N-daily-temperature-precipitation")
def read_bz_excel(path):
    # fname = path.stem.replace(re.compile(r"-+"), "-")
    path_meta = meta_pattern.match(path.stem).groupdict()
    # original_id, station_name = fname.split("-", maxsplit=1)
    # station_name = station_name.replace("-", " ")
    # original_id = original_id.upper().strip()
    station_name = path_meta["station_name_it"].strip()
    original_id = path_meta["original_id"].upper().strip()

    meta = get_meta(path)
    meta["original_id"] = original_id
    meta["station_name"] = station_name

    data = pd.read_excel(
        path,
        skiprows=13,
        usecols=[2, 4, 5],
        names=["date", "T_MIN", "T_MAX"],
        na_values="---",
        dtype={"T_MIN": "float", "T_MAX": "float"},
        skipfooter=1,
    )
    data = pl.from_pandas(data).with_columns(
        pl.col("date").str.to_date(format=r"%d.%m.%Y"),
        pl.lit(original_id).alias("original_id").str.to_uppercase().str.strip_chars(),
        pl.lit(station_name).alias("station_name"),
    )
    return data, meta

In [ ]:
for path in xlsx_files:
    try:
        read_bz_excel(path)
    except Exception as e:
        print(f"Could not open {path} because {e}")
        # path.unlink()

In [14]:
all_things = [read_bz_excel(path) for path in excels_path.glob("*.xlsx")]

In [15]:
data, meta = zip(*all_things)

In [16]:
data = pl.concat(data, how="vertical")
meta = pl.concat(
    [pl.from_pandas(pd.DataFrame(m).transpose()) for m in meta], how="vertical"
)

In [17]:
meta = meta.with_columns(
    pl.col("x").str.to_integer(),
    pl.col("y").str.to_integer(),
    pl.col("elevation").str.to_integer(),
)

In [18]:
meta.write_csv(base / "xlsx" / "meta.csv")

In [19]:
data.write_parquet(base / "xlsx" / "data.parquet")